In [35]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [36]:
num_input = 3
num_output = 3

num_hidden1 = 16
num_hidden2 = 16

my_rmse_list = []

In [37]:

session = tf.Session()

x = tf.placeholder(tf.float32, shape=[None, num_input])

y_true = tf.placeholder(tf.float32, shape=[None, num_output], name='y_true')
y_pred = tf.placeholder(tf.float32, shape=[None, num_output])


In [38]:
data_csv = pd.read_csv('../data.csv')
X = data_csv[['in1','in2','in3']]
y = data_csv[['out1','out2','out3']]
#X, y
print (type(X), type(y))
#y

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.frame.DataFrame'>


In [39]:

X_a = X.values
y_a = y.values
y_dumb = X_a*2
y_x = X_a

In [40]:

def cost_func(layer, my_y_true):
    ret = tf.sqrt(tf.reduce_mean(tf.square(layer - my_y_true)))
    
    return ret

In [41]:
def nn_model_fn(features, labels, mode):
    global my_rmse_list
    # Neural Network Structure
    # Input Layer
    input_layer = tf.reshape(features["x"],[-1, 3]) 
    
    # Hidden Layer #1
    h1 = tf.layers.dense(inputs=input_layer, 
                         units=num_hidden1,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Hidden Layer #2
    h2 = tf.layers.dense(inputs=h1, 
                         units=num_hidden2,
                         use_bias=True,
                         activation=tf.nn.relu)

    # Output Layer
    output_layer = tf.layers.dense(inputs=h2, 
                         units=num_output,
                         use_bias=True,
                         activation=None)
    predictions = {
        "output" : output_layer
    }
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)
    
    cost = cost_func(output_layer, labels)
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-2)
        
        train_op = optimizer.minimize(
                    loss=cost,
                    global_step=tf.train.get_global_step())
        #my_rmse_list += [cost.eval]
        return tf.estimator.EstimatorSpec(mode=mode, loss=cost, train_op=train_op)
    
    eval_metric_ops = {
        "error":tf.metrics.mean_squared_error(
        labels=labels, predictions=predictions["output"])
    }
    return tf.estimator.EstimatorSpec(mode=mode, loss=cost, eval_metric_ops=eval_metric_ops)

In [42]:
my_nn_esti = tf.estimator.Estimator(model_fn=nn_model_fn, model_dir="./sav")

tensors_to_log = {"RMSE": "output"}
rmse_list = tf.train.LoggingTensorHook(
    tensors=tensors_to_log, every_n_iter=1000)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': './sav', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x00000197BB94A5C0>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [45]:
train_input_fn = tf.estimator.inputs.numpy_input_fn(
                    #x={"in1" : X_a[:,0], "in2":X_a[:,1], "in3":X_a[:,2]},
                    x={"x":X_a},
                    #y=y_a,
                    y=y_x,
                    batch_size=1000,
                    num_epochs=None,
                    shuffle=True)
my_nn_esti.train(
                input_fn=train_input_fn,
                steps=10000)


INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from ./sav\model.ckpt-1002
INFO:tensorflow:Saving checkpoints for 1003 into ./sav\model.ckpt.
INFO:tensorflow:loss = 0.01891901353341219, step = 1003
INFO:tensorflow:global_step/sec: 167.469
INFO:tensorflow:loss = 0.0037279010976840276, step = 1103 (0.598 sec)
INFO:tensorflow:global_step/sec: 176.977
INFO:tensorflow:loss = 0.008662373713428298, step = 1203 (0.565 sec)
INFO:tensorflow:global_step/sec: 173.101
INFO:tensorflow:loss = 0.002490810805566895, step = 1303 (0.578 sec)
INFO:tensorflow:global_step/sec: 176.429
INFO:tensorflow:loss = 0.0026691206351932583, step = 1403 (0.567 sec)
INFO:tensorflow:global_step/sec: 179.518
INFO:tensorflow:loss = 0.0053847598538215415, step = 1503 (0.557 sec)
INFO:tensorflow:global_step/sec: 160.628
INFO:tensorflow:loss = 0.004775743241869562, step = 1603 (0.622 sec)
INFO:tensorflow:global_step/sec: 176.816
INFO:tensorflow:loss = 0.005419029170400484, step = 1703 (0.566 s

INFO:tensorflow:global_step/sec: 176.235
INFO:tensorflow:loss = 0.005093287347892505, step = 8403 (0.567 sec)
INFO:tensorflow:global_step/sec: 177.947
INFO:tensorflow:loss = 0.0027616598186087768, step = 8503 (0.562 sec)
INFO:tensorflow:global_step/sec: 181.117
INFO:tensorflow:loss = 0.004987971673054186, step = 8603 (0.552 sec)
INFO:tensorflow:global_step/sec: 176.405
INFO:tensorflow:loss = 0.001930671816852282, step = 8703 (0.567 sec)
INFO:tensorflow:global_step/sec: 175.037
INFO:tensorflow:loss = 0.008574624199617579, step = 8803 (0.571 sec)
INFO:tensorflow:global_step/sec: 174.028
INFO:tensorflow:loss = 0.0038564913789447177, step = 8903 (0.575 sec)
INFO:tensorflow:global_step/sec: 176.165
INFO:tensorflow:loss = 0.005695777847847951, step = 9003 (0.568 sec)
INFO:tensorflow:global_step/sec: 178.711
INFO:tensorflow:loss = 0.006405121729190184, step = 9103 (0.559 sec)
INFO:tensorflow:global_step/sec: 172.791
INFO:tensorflow:loss = 0.008697770672125503, step = 9203 (0.580 sec)
INFO:ten

In [ ]:
print (my_rmse_list)